In [6]:
state = 'LA'
level = 'blockgroup'
district_type = 'SH'  
minority = 'Black'
deviation = 0.10      # 0.10 means 10% means +/-5%

In [7]:
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.optimization import SingleMetricOptimizer, Gingleator
from gerrychain.tree import recursive_seed_part
from functools import partial
import pandas as pd
import json
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import random

random.seed(2024)

In [8]:
filepath = 'C://districting-data-2020//'
filename = state + '_' + level + ".json"
G = Graph.from_json(filepath+filename)

for i in G.nodes:
    G.nodes[i]['TOTPOP'] = G.nodes[i]['P0010001']

from number_of_districts import number_of_districts
k = number_of_districts[state,district_type]

total_population = sum( G.nodes[i]['TOTPOP'] for i in G.nodes )
ideal_population = total_population / k
import math
L = math.ceil( ideal_population * (1-deviation/2) )
U = math.floor( ideal_population * (1+deviation/2) )

print("Effectively using k, L, U =",k,L,U)

Effectively using k, L, U = 105 42142 46577


In [9]:
from district import get_census_codes
codes = get_census_codes(minority)
        
for i in G.nodes:
    G.nodes[i]['VAP'] = G.nodes[i]['P0030001']
    G.nodes[i]['MVAP'] = sum( G.nodes[i][code] for code in codes )

In [13]:
from complete import recursive_bipartition_heuristic
import networkx as nx
DG = nx.DiGraph(G)
DG._k = k
DG._L = L
DG._U = U
districts = recursive_bipartition_heuristic(DG)
labeling = { i : j for j in range(k) for i in districts[j] }

Using one split county, attempting to bipartition cluster into sizes: 52 53
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-16


C:\Users\buchanan\OneDrive - Oklahoma A and M System\Code\polsby_popper_polished\Polsby_Popper_optimization\src\revision\district.py:179: DeprecationWarning: Calling quicksum on a tupledict is deprecated, use .sum() instead.
  m.addConstr( gp.quicksum( is_split ) <= split_counties_limit )


Using one split county, attempting to bipartition cluster into sizes: 26 27
Using one split county, attempting to bipartition cluster into sizes: 13 14
Using one split county, attempting to bipartition cluster into sizes: 7 7
Using one split county, attempting to bipartition cluster into sizes: 3 4
Using one split county, attempting to bipartition cluster into sizes: 2 2
Using one split county, attempting to bipartition cluster into sizes: 1 1
Using one split county, attempting to bipartition cluster into sizes: 1 1
Using one split county, attempting to bipartition cluster into sizes: 1 2
Using one split county, attempting to bipartition cluster into sizes: 1 1
Using one split county, attempting to bipartition cluster into sizes: 3 4
Using one split county, attempting to bipartition cluster into sizes: 2 2
Using one split county, attempting to bipartition cluster into sizes: 1 1
Using one split county, attempting to bipartition cluster into sizes: 1 1
Using one split county, attempting

In [16]:
chain_updaters = {
"population": updaters.Tally("TOTPOP", alias="population"),
"VAP": updaters.Tally("VAP"),
"MVAP": updaters.Tally("MVAP")
}

initial_partition = Partition(graph=G,
    assignment=labeling,
updaters=chain_updaters
)

proposal = partial(
    proposals.recom,
    pop_col="TOTPOP",
    pop_target=ideal_population,
    epsilon=deviation/2,
    node_repeats=1
)

constraints = constraints.within_percent_of_ideal_population(initial_partition, deviation/2)

In [17]:
gingles = Gingleator(
    proposal,
    constraints,
    initial_partition,
    minority_pop_col="MVAP",
    total_pop_col="VAP",
    score_function=Gingleator.reward_partial_dist
)

In [38]:
from metrics import number_of_counties_split, number_of_county_splits, average_polsby_popper

0


In [45]:
total_steps = 100000

# Short Bursts
max_scores_sb = np.zeros(total_steps)
scores_sb = np.zeros(total_steps)
pp = np.zeros(total_steps)
s1 = np.zeros(total_steps)
s2 = np.zeros(total_steps)
for i, part in enumerate(gingles.short_bursts(10, 10000, with_progress_bar=True)):
    max_scores_sb[i] = gingles.best_score
    scores_sb[i] = gingles.score(part)
    districts = [ list() for j in range(k) ]
    for v in G.nodes:
        j = part.assignment[v]
        districts[j].append(v)
    s1[i] = number_of_counties_split(G, districts)
    s2[i] = number_of_county_splits(G, districts)
    pp[i] = average_polsby_popper(G, districts)

# # Simulated Annealing
# max_scores_anneal = np.zeros(total_steps)
# scores_anneal = np.zeros(total_steps)
# for i, part in enumerate(
#     gingles.simulated_annealing(
#         total_steps,
#         gingles.jumpcycle_beta_function(1000, 4000),
#         beta_magnitude=500,
#         with_progress_bar=True
#     )
# ):
#     max_scores_anneal[i] = gingles.best_score
#     scores_anneal[i] = gingles.score(part)

# # Tilted Runs
# max_scores_tilt = np.zeros(total_steps)
# scores_tilt = np.zeros(total_steps)
# for i, part in enumerate(gingles.tilted_run(total_steps, 0.125, with_progress_bar=True)):
#     max_scores_tilt[i] = gingles.best_score
#     scores_tilt[i] = gingles.score(part)

100%|██████████████████████████████████████████████████████████████████████████| 100000/100000 [50:52<00:00, 32.76it/s]


In [46]:
print("i gingles pp s1 s2")
for i in range(total_steps):
    if i%100==0:
        print(i,round(scores_sb[i], 4), round(pp[i],4), round(s1[i]), round(s2[i]))

i gingles pp s1 s2
0 15.4951 0.3472 38 104
100 16.4952 0.2971 45 143
200 16.4993 0.2867 49 150
300 18.4967 0.2695 55 170
400 18.4969 0.2643 57 174
500 18.4969 0.2661 53 171
600 19.4946 0.2602 53 172
700 21.4824 0.2481 54 181
800 21.4972 0.2496 55 177
900 22.4476 0.2471 54 184
1000 22.4544 0.2364 55 185
1100 22.4544 0.2288 55 193
1200 23.4448 0.2303 57 198
1300 23.4557 0.2431 58 198
1400 23.4557 0.2444 58 193
1500 24.4473 0.2431 56 190
1600 24.4473 0.2445 56 189
1700 25.4237 0.2399 55 193
1800 25.4379 0.2332 58 192
1900 25.4666 0.2386 58 190
2000 25.4666 0.2401 58 189
2100 25.4666 0.2452 57 190
2200 25.4666 0.2342 56 193
2300 25.4666 0.2337 57 192
2400 25.4666 0.2372 58 188
2500 25.4677 0.2442 59 189
2600 25.482 0.2456 58 191
2700 25.482 0.2443 57 189
2800 25.482 0.2419 58 186
2900 25.482 0.2409 58 190
3000 25.482 0.2449 57 190
3100 25.482 0.2519 55 184
3200 25.482 0.2458 55 186
3300 25.482 0.2383 57 193
3400 25.482 0.2329 56 191
3500 25.482 0.2295 56 192
3600 25.482 0.2352 55 188
3700 

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
plt.plot(max_scores_sb, label="Short Bursts")
plt.plot(max_scores_anneal, label="Simulated Annealing")
plt.plot(max_scores_tilt, label="Tilted Run")
plt.xlabel("Steps", fontsize=20)
plt.ylabel("Max Score Observered", fontsize=20)
plt.legend()
plt.show()

In [22]:
max( max_scores_sb )

34.4890764209474